In [1]:
import numpy as np
import random
from tqdm import tqdm

In [2]:
M = np.load("ProbMatrix.npy")

In [3]:
class card:
    def __init__(self, coat, face, number):
        self.coat = coat
        self.face = face
        self.number = number

In [4]:
class Player:
    def __init__(self):
        self.hand = []
        self.score = 0
        self.passed = False
        self.out = False
    def nextMove(self, deck, play):
        if play == "pass":
            self.passed = True
            return "Pass"
        ind = random.randint(0, len(deck) - 1)
        card = deck[ind]
        deck.pop(ind)
        self.score += card.number
        self.hand.append(card)
        if self.score > 21:
            self.out = True
            return card
        return card
        
    def printHand(self):
        if len(self.hand) == 0:
            return None
        for i in range(len(self.hand) - 1):
            if self.hand[i].coat in ["Ace", "Jack", "Queen", "King"]:
                print(self.hand[i].coat[0], end = " ")
            else:
                print(self.hand[i].coat, end = " ")
        if self.hand[-1].coat in ["Ace", "Jack", "Queen", "King"]:
            print(self.hand[-1].coat[0], end = " ")
        else:
            print(self.hand[-1].coat, end = " ")
        if self.passed == True:
            print("# | " + str(self.score), end = " ")
        elif self.out == True:
            print("X", end = " ")
        else:
            print(" | " + str(self.score), end = " ")

In [5]:
class Computer:
    def __init__(self):
        self.hand = []
        self.score = 0
        self.passed = False
        self.out = False
    def nextMove(self, others, deck):
        if self.score <= 11:
            ind = random.randint(0, len(deck) - 1)
            card = deck[ind]
            deck.pop(ind)
            self.score += card.number
            self.hand.append(card)
            return card
        if self.score == 21:
            if self.passed != True:
                self.passed = True
                return "Pass"
        playing_deck = []
        out_deck = []
        passed_deck = []
        for i in range(0, len(others)):
            if others[i].passed:
                passed_deck.append(others[i])
            elif others[i].out:
                out_deck.append(others[i])
            else:
                playing_deck.append(others[i])
        not_out_deck = playing_deck
        not_out_deck.extend(passed_deck)
        for i in range(0, len(not_out_deck)):
            if not_out_deck[i].score > self.score:
                ind = random.randint(0, len(deck) - 1)
                card = deck[ind]
                deck.pop(ind)
                self.score += card.number
                self.hand.append(card)
                if self.score == 21:
                    self.passed = True
                if self.score > 21:
                    self.out = True
                return card
            if not_out_deck[i].score == self.score:
                if len(not_out_deck[i].hand) < len(self.hand):
                    ind = random.randint(0, len(deck) - 1)
                    card = deck[ind]
                    deck.pop(ind)
                    self.score += card.number
                    self.hand.append(card)
                    if self.score == 21:
                        self.passed = True
                    if self.score > 21:
                        self.out = True
                    return card
                if len(not_out_deck[i].hand) == len(self.hand):
                    scoreSelf = []
                    for c in self.hand:
                        scoreSelf.append(c.number)
                    scoreOther = []
                    for c in not_out_deck[i].hand:
                        scoreOther.append(c.number)
                    scoreSelf = sorted(scoreSelf, reverse = True)
                    scoreOther = sorted(scoreOther, reverse = True)
                    for j in range(0, len(scoreSelf)):
                        if scoreSelf[j] > scoreOther[j]:
                            self.passed = True
                            return "Pass"
                        if scoreSelf[j] < scoreOther[j]:
                            ind = random.randint(0, len(deck) - 1)
                            card = deck[ind]
                            deck.pop(ind)
                            self.score += card.number
                            self.hand.append(card)
                            if self.score == 21:
                                self.passed = True
                            if self.score > 21:
                                self.out = True
                            return card
        playing_deck = []
        done_deck = []
        for i in range(0, len(others)):
            if others[i].passed or others[i].out:
                done_deck.append(others[i])
            else:
                playing_deck.append(others[i])
        # If no score is above the current hand, it will reach here.
        if len(playing_deck) == 0:
            self.passed = True
            return "Pass"
        P = []
        for p in playing_deck:
            P.append(M[self.score - 1][p.score - 1])
        p = max(P)
        if random.random() < 1 - p:
            ind = random.randint(0, len(deck) - 1)
            card = deck[ind]
            deck.pop(ind)
            self.score += card.number
            self.hand.append(card)
            if self.score > 21:
                self.out = True
                return card
            if self.score == 21:
                self.passed = True
                return card
            return card
        else:
            self.passed = True
            return "Pass"
            
    def printHand(self):
        if len(self.hand) == 0:
            return None
        for i in range(len(self.hand) - 1):
            if self.hand[i].coat in ["Ace", "Jack", "Queen", "King"]:
                print(self.hand[i].coat[0], end = " ")
            else:
                print(self.hand[i].coat, end = " ")
        if self.hand[-1].coat in ["Ace", "Jack", "Queen", "King"]:
            print(self.hand[-1].coat[0], end = " ")
        else:
            print(self.hand[-1].coat, end = " ")
        if self.passed == True:
            print("# | " + str(self.score), end = " ")
        elif self.out == True:
            print("X", end = " ")
        else:
            print(" | " + str(self.score), end = " ")

In [6]:
def BlackJack():
    deck = []
    for i in range(0, 13):
        if i == 1:
            deck.append(card("Ace", "Spades", i))
            deck.append(card("Ace", "Clubs", i))
            deck.append(card("Ace", "Heart", i))
            deck.append(card("Ace", "Diamond", i))
        if i <= 10 and i > 1:
            deck.append(card(str(i), "Spades", i))
            deck.append(card(str(i), "Clubs", i))
            deck.append(card(str(i), "Heart", i))
            deck.append(card(str(i), "Diamond", i))
        if i == 11:
            deck.append(card("Jack", "Spades", 10))
            deck.append(card("Jack", "Clubs", 10))
            deck.append(card("Jack", "Heart", 10))
            deck.append(card("Jack", "Diamond", 10))
        if i == 12:
            deck.append(card("Queen", "Spades", 10))
            deck.append(card("Queen", "Clubs", 10))
            deck.append(card("Queen", "Heart", 10))
            deck.append(card("Queen", "Diamond", 10))
        if i == 13:
            deck.append(card("King", "Spades", 10))
            deck.append(card("King", "Clubs", 10))
            deck.append(card("King", "Heart", 10))
            deck.append(card("King", "Diamond", 10))
    n = int(input("Number of players : "))
    Players = []
    Types = []
    Hands = []
    for i in range(0, n):
        a = input("Player " + str(i + 1) + " Name : ")
        b = input("Player " + str(i + 1) + " Type (Player/Computer)(P/C) : ")
        Players.append(a)
        Types.append(b.lower())
        if b.lower() == "p":
            Hands.append(Player())
        else:
            Hands.append(Computer())
    passed = []
    out = []
    passedPlayers = []
    curr = 0
    while True:
        index = curr%(n)
        others = Hands[:index]
        try:
            others.extend(Hands[index + 1:])
        except:
            x = 5
        if len(passed) + len(out) == n:
            break
        if Types[index] == "c":
            if Hands[index].passed == True:
                curr += 1
                x = 5
            elif Hands[index].out == True:
                curr += 1
                x = 5
            else:
                for i in range(0, len(Players)):
                    print(Players[i] + " Hand : ", end = " ")
                    Hands[i].printHand()
                    print("")
                N = Hands[index].nextMove(others, deck)
                if N == "Pass":
                    print(Players[index] + " passes.")
                    print("")
                    passed.append(Hands[index])
                    passedPlayers.append(Players[index])
                    curr += 1
                else:
                    print(Players[index] + " hits and gets " + N.coat + " of " + N.face + ".")
                    print("")
                    if Hands[index].out == True:
                        out.append(Hands[index])
                    if Hands[index].passed == True:
                        passed.append(Hands[index])
                        passedPlayers.append(Players[index])
                    curr += 1
        if Types[index] == "p":
            if Hands[index].passed == True:
                curr += 1
                x = 5
            elif Hands[index].out == True:
                curr += 1
                x = 5
            else:
                for i in range(0, len(Players)):
                    print(Players[i] + " Hand : ", end = " ")
                    Hands[i].printHand()
                    print("")
                print(Players[index] + "'s Turn")
                play = input("Pass or Play?").lower()
                N = Hands[index].nextMove(deck, play)
                if N == "Pass":
                    print(Players[index] + " passes.")
                    print("")
                    passed.append(Hands[index])
                    passedPlayers.append(Players[index])
                    curr += 1
                else:
                    print(Players[index] + " hits and gets " + N.coat + " of " + N.face + ".")
                    print("")
                    if Hands[index].out == True:
                        out.append(Hands[index])
                    if Hands[index].passed == True:
                        passed.append(Hands[index])
                        passedPlayers.append(Players[index])
                    curr += 1
        
    maxi = 0
    maxDeck = None
    winner = None
    for i in range(0, len(passed)):
        if passed[i].score > maxi:
            maxi = passed[i].score
            winner = passedPlayers[i]
            maxDeck = passed[i].hand
        if passed[i].score == maxi:
            if len(passed[i].hand) < len(maxDeck):
                maxi = passed[i].score
                winner = passedPlayers[i]
                maxDeck = passed[i].hand
            elif len(passed[i].hand) == len(maxDeck):
                scoreNew = []
                for c in passed[i].hand:
                    scoreNew.append(c.number)
                scoreMax = []
                for c in maxDeck:
                    scoreMax.append(c.number)
                scoreNew = sorted(scoreNew, reverse = True)
                scoreMax = sorted(scoreMax, reverse = True)
                for j in range(0, len(scoreNew)):
                    if scoreNew[j] > scoreMax[j]:
                        maxi = passed[i].score
                        winner = passedPlayers[i]
                        maxDeck = passed[i].hand
                        break
                    if scoreNew[j] < scoreMax[j]:
                        break
                        
    print("###")
    print(winner + " Wins!!!")

### This is to see if all the computers are equally skilled. So, I will make n computers play against each other, and compare how many each won.

In [7]:
def BlackJackTrialOne(n):
    deck = []
    for i in range(0, 13):
        if i == 1:
            deck.append(card("Ace", "Spades", i))
            deck.append(card("Ace", "Clubs", i))
            deck.append(card("Ace", "Heart", i))
            deck.append(card("Ace", "Diamond", i))
        if i <= 10 and i > 1:
            deck.append(card(str(i), "Spades", i))
            deck.append(card(str(i), "Clubs", i))
            deck.append(card(str(i), "Heart", i))
            deck.append(card(str(i), "Diamond", i))
        if i == 11:
            deck.append(card("Jack", "Spades", 10))
            deck.append(card("Jack", "Clubs", 10))
            deck.append(card("Jack", "Heart", 10))
            deck.append(card("Jack", "Diamond", 10))
        if i == 12:
            deck.append(card("Queen", "Spades", 10))
            deck.append(card("Queen", "Clubs", 10))
            deck.append(card("Queen", "Heart", 10))
            deck.append(card("Queen", "Diamond", 10))
        if i == 13:
            deck.append(card("King", "Spades", 10))
            deck.append(card("King", "Clubs", 10))
            deck.append(card("King", "Heart", 10))
            deck.append(card("King", "Diamond", 10))
    Players = []
    Hands = []
    for i in range(0, n):
        a = "Player " + str(i + 1)
        Players.append(a)
        Hands.append(Computer())
    passed = []
    out = []
    passedPlayers = []
    curr = 0
    while True:
        index = curr%(n)
        others = Hands[:index]
        try:
            others.extend(Hands[index + 1:])
        except:
            x = 5
        if len(passed) + len(out) == n:
            break
        if Hands[index].passed == True:
            curr += 1
            x = 5
        elif Hands[index].out == True:
            curr += 1
            x = 5
        else:
            N = Hands[index].nextMove(others, deck)
            if N == "Pass":
                passed.append(Hands[index])
                passedPlayers.append(Players[index])
                curr += 1
            else:
                if Hands[index].out == True:
                    out.append(Hands[index])
                if Hands[index].passed == True:
                    passedPlayers.append(Players[index])
                    passed.append(Hands[index])
                curr += 1
        
    maxi = 0
    maxDeck = None
    winner = None
    if len(passed) == 1:
        winner = passedPlayers[0]
        return winner
    for i in range(0, len(passed)):
        if passed[i].score > maxi:
            maxi = passed[i].score
            winner = passedPlayers[i]
            maxDeck = passed[i].hand
        if passed[i].score == maxi:
            if len(passed[i].hand) < len(maxDeck):
                maxi = passed[i].score
                winner = passedPlayers[i]
                maxDeck = passed[i].hand
            elif len(passed[i].hand) == len(maxDeck):
                scoreNew = []
                for c in passed[i].hand:
                    scoreNew.append(c.number)
                scoreMax = []
                for c in maxDeck:
                    scoreMax.append(c.number)
                scoreNew = sorted(scoreNew, reverse = True)
                scoreMax = sorted(scoreMax, reverse = True)
                for j in range(0, len(scoreNew)):
                    if scoreNew[j] > scoreMax[j]:
                        maxi = passed[i].score
                        winner = passedPlayers[i]
                        maxDeck = passed[i].hand
                        break
                    if scoreNew[j] < scoreMax[j]:
                        break
                        
    return winner

def BlackJackTrial(n, m):
    D = {}
    for i in range(0, m):
        D["Player " + str(i + 1)] = 0
    for i in range(0, n):
        winner = BlackJackTrialOne(m)
        D[winner] += 1
    return D

In [8]:
BlackJackTrial(1000, 5)

{'Player 1': 191,
 'Player 2': 193,
 'Player 3': 214,
 'Player 4': 180,
 'Player 5': 222}

In [9]:
BlackJack()

Number of players :  2
Player 1 Name :  Donald Trump
Player 1 Type (Player/Computer)(P/C) :  C
Player 2 Name :  Joe Biden
Player 2 Type (Player/Computer)(P/C) :  C


Donald Trump Hand :  
Joe Biden Hand :  
Donald Trump hits and gets 2 of Clubs.

Donald Trump Hand :  2  | 2 
Joe Biden Hand :  
Joe Biden hits and gets 4 of Spades.

Donald Trump Hand :  2  | 2 
Joe Biden Hand :  4  | 4 
Donald Trump hits and gets 8 of Heart.

Donald Trump Hand :  2 8  | 10 
Joe Biden Hand :  4  | 4 
Joe Biden hits and gets Queen of Clubs.

Donald Trump Hand :  2 8  | 10 
Joe Biden Hand :  4 Q  | 14 
Donald Trump hits and gets Ace of Heart.

Donald Trump Hand :  2 8 A  | 11 
Joe Biden Hand :  4 Q  | 14 
Joe Biden hits and gets Ace of Clubs.

Donald Trump Hand :  2 8 A  | 11 
Joe Biden Hand :  4 Q A  | 15 
Donald Trump hits and gets Jack of Diamond.

Donald Trump Hand :  2 8 A J  | 21 
Joe Biden Hand :  4 Q A  | 15 
Joe Biden hits and gets 5 of Spades.

Donald Trump Hand :  2 8 A J  | 21 
Joe Biden Hand :  4 Q A 5  | 20 
Donald Trump passes.

Donald Trump Hand :  2 8 A J # | 21 
Joe Biden Hand :  4 Q A 5  | 20 
Joe Biden hits and gets 3 of Diamond.

###
Donald Trump Wi